In [1]:
import warnings
warnings.filterwarnings('ignore')
import string
import re
from unicodedata import normalize
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import LSTM,Dense,Embedding,RepeatVector,TimeDistributed
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.translate.bleu_score import corpus_bleu
import pandas as pd
from string import punctuation
import matplotlib.pyplot as plt
from IPython.display import Markdown, display

def printmd(string):
    # Print with Markdowns    
    display(Markdown(string))

In [2]:
train = pd.read_csv('preprocessed/train_matched_combined.csv')
dev_en = pd.read_csv('preprocessed/cleaned_dev_en.csv')
dev_tcn = pd.read_csv('preprocessed/cleaned_dev_tcn.csv')

In [3]:
train

,Original,Translated,Matched,Cosine_sim
0,耐吉 官方 男子 足球 長 褲 新款 標準 型 拒水 拉鏈 褲腳,Niji official men's football trousers new stan...,trousers,0.555
1,火影忍者 六道 鸣人 睡衣 卡卡 西宇智 波佐助 恤 二次元 动漫 短袖 衣服,Naruto Six Naruto Pajamas Caka Xiuxi Poto Asso...,flat sheet blanket cotton,0.000
2,小拇指 鞋坊 金標 貝殼頭 白紅時 尚 男女 鞋板 鞋 時尚 情侶 鞋 男鞋 女鞋,Small thumb shoes gold standard shell head whi...,fashion sports shoes women s shoes,0.784
3,居家 大 推款 記憶 棉枕護 勁 頸 椎 枕修 復 脊椎 治頸椎 專用 枕頭 女孕婦 失眠 ...,Home Big Push Memory Cotton Pillow Care Cervic...,flat sheet blanket cotton,0.000
4,炫彩 細口手 沖 壺桃紅 藍 黃,Colorful thin mouthful pot pot red blue yellow,mini pot extra small pot taho pot,0.526
5,個性 不 規則 拼接 抽繩 設計 圓領 短袖 恤,Personality irregular splicing rope design rou...,unisex t shirt round neck short sleeve shirt,0.500
6,迪 最新 秋冬 尖頭 及 裸 靴 牛皮 與 羊皮 色碼 精品 女鞋 現拍 增高鞋 透氣 真皮...,Di latest autumn and winter pointers and bare ...,leather shoes,0.539
7,胡桃 裡 家居 旗艦 店北 歐鐵藝 餐椅 現代 家用 簡約 靠背 休 閑 椅化妝椅 創意書 ...,Wooden wrought iron restaurants in the norther...,high back chair office chair conference chair ...,0.579
8,二手 丹寧 寬褲,Second-hand Danning Wide Pants,on hand,0.316
9,洗紅 牛仔 外套,Washing red denim jacket,denim jacket,0.707


In [4]:
dev_en

,translation_output
0,oppo phone case soft rabbit silicone case
1,soft coating car wax strong water watt
2,low sugar mango dry be the royal
3,the culture japan imported round top space cra...
4,hello kitty sandals shoes white red children
5,thunderbird lt japanese wood grain table mat x...
6,the sand kid dvd vol
7,anacomda python titan series ssd drive
8,ifairies shoulder messenger bag shoulder bag
9,the farm mountain bitter gourd tea pack box


In [5]:
dev_tcn

,text,split,product_title_clean,product_title_processed
0,OPPO A75 A75s A73 手机壳 软壳 挂绳壳 大眼兔硅胶壳,private,手机壳软壳挂绳壳大眼兔硅胶壳,手 机 壳 软 壳 挂 绳 壳 大眼 兔 硅 胶 壳
1,SOFT 99 鍍膜車蠟(強力撥水型),private,鍍膜車蠟強力撥水型,鍍膜 車蠟 強力 撥水型
2,低糖芒果乾 250g 臻御行,private,低糖芒果乾臻御行,低糖 芒果乾 臻 御行
3,＊小徑文化＊日本進口ROUND TOP space craft - diamond (SC-...,private,小徑文化日本進口,小徑 文化 日本 進口
4,Hello Kitty 凱蒂貓 KITTY 涼鞋 童鞋 白/紅色 小童 no739,private,凱蒂貓涼鞋童鞋白紅色小童,凱蒂貓 涼鞋 童鞋 白 紅色 小童
5,雷鳥 LT-234 日式木紋桌墊 45 x 60 cm / 片,private,雷鳥日式木紋桌墊片,雷鳥 日式 木紋 桌 墊片
6,狂砂小子DVD VOL-08,private,狂砂小子,狂砂 小子
7,ANACOMDA 巨蟒 T1 泰坦系列 120GB SSD固態硬碟,private,巨蟒泰坦系列固態硬碟,巨蟒 泰坦 系列 固態 硬碟
8,iFairies 單肩斜背包側背包肩背包【49146】,private,單肩斜背包側背包肩背包,單肩 斜 背包 側背包 肩 背包
9,大雪山農場 山苦瓜茶20包/盒,private,大雪山農場山苦瓜茶包盒,大雪山 農場 山 苦瓜 茶包 盒


In [6]:
df1 = train[['Translated', 'Original']]
df1.columns = ['translation_output',	'product_title_processed']
df2 =  pd.concat([dev_en['translation_output'], dev_tcn['product_title_processed']], axis=1)
df = pd.concat([df1, df2], axis=0)
df = df.dropna(how='any', axis=0)
df.columns = ['translation_output',	'text']

In [7]:
df

,translation_output,text
0,Niji official men's football trousers new stan...,耐吉 官方 男子 足球 長 褲 新款 標準 型 拒水 拉鏈 褲腳
1,Naruto Six Naruto Pajamas Caka Xiuxi Poto Asso...,火影忍者 六道 鸣人 睡衣 卡卡 西宇智 波佐助 恤 二次元 动漫 短袖 衣服
2,Small thumb shoes gold standard shell head whi...,小拇指 鞋坊 金標 貝殼頭 白紅時 尚 男女 鞋板 鞋 時尚 情侶 鞋 男鞋 女鞋
3,Home Big Push Memory Cotton Pillow Care Cervic...,居家 大 推款 記憶 棉枕護 勁 頸 椎 枕修 復 脊椎 治頸椎 專用 枕頭 女孕婦 失眠 ...
4,Colorful thin mouthful pot pot red blue yellow,炫彩 細口手 沖 壺桃紅 藍 黃
5,Personality irregular splicing rope design rou...,個性 不 規則 拼接 抽繩 設計 圓領 短袖 恤
6,Di latest autumn and winter pointers and bare ...,迪 最新 秋冬 尖頭 及 裸 靴 牛皮 與 羊皮 色碼 精品 女鞋 現拍 增高鞋 透氣 真皮...
7,Wooden wrought iron restaurants in the norther...,胡桃 裡 家居 旗艦 店北 歐鐵藝 餐椅 現代 家用 簡約 靠背 休 閑 椅化妝椅 創意書 ...
8,Second-hand Danning Wide Pants,二手 丹寧 寬褲
9,Washing red denim jacket,洗紅 牛仔 外套


In [ ]:
def create_tokenizer(lines):
    # fit a tokenizer
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer
 
def max_len(lines):
    # max sentence length
    return max(len(line.split()) for line in lines)

def encode_sequences(tokenizer, length, lines):
    # encode and pad sequences
    X = tokenizer.texts_to_sequences(lines) # integer encode sequences
    X = pad_sequences(X, maxlen=length, padding='post') # pad sequences with 0 values
    return X
 
def encode_output(sequences, vocab_size):
    # one hot encode target sequence
    ylist = list()
    for sequence in sequences:
        encoded = to_categorical(sequence, num_classes=vocab_size)
        ylist.append(encoded)
    y = np.array(ylist)
    y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
    return y
 
# Prepare target tokenizer
tar_tokenizer = create_tokenizer(df['translation_output'])
tar_vocab_size = len(tar_tokenizer.word_index) + 1
tar_length = max_len(df['translation_output'])
printmd(f'\nTarget Vocabulary Size: {tar_vocab_size}')
printmd(f'Target Max Length: {tar_length}')

# Prepare source tokenizer
src_tokenizer = create_tokenizer(df['text'])
src_vocab_size = len(src_tokenizer.word_index) + 1
src_length = max_len(df['text'])
printmd(f'\nSource Vocabulary Size: {src_vocab_size}')
printmd(f'Source Max Length: {src_length}\n')
 
train, test = df[:-1000], df[-1000:]

# Prepare training data
trainX = encode_sequences(src_tokenizer, src_length, train['text'])
trainY = encode_sequences(tar_tokenizer, tar_length, train['translation_output'])
trainY = encode_output(trainY, tar_vocab_size)

# Prepare test data
testX = encode_sequences(src_tokenizer, src_length, test['text'])
testY = encode_sequences(tar_tokenizer, tar_length, test['translation_output'])
testY = encode_output(testY, tar_vocab_size)


Target Vocabulary Size: 42688

Target Max Length: 141


Source Vocabulary Size: 158697

Source Max Length: 88


In [ ]:
trainY

In [ ]:
def create_model(src_vocab, tar_vocab, src_timesteps, tar_timesteps, n_units):
    # Create the model
    model = Sequential()
    model.add(Embedding(src_vocab, n_units, input_length=src_timesteps, mask_zero=True))
    model.add(LSTM(n_units))
    model.add(RepeatVector(tar_timesteps))
    model.add(LSTM(n_units, return_sequences=True))
    model.add(TimeDistributed(Dense(tar_vocab, activation='softmax')))
    return model
 
# Create model
model = create_model(src_vocab_size, tar_vocab_size, src_length, tar_length, 256)
model.compile(optimizer='adam', loss='categorical_crossentropy')

history = model.fit(trainX, trainY, epochs=500, batch_size=64, validation_split=0.1, verbose=1, 
                    callbacks=[EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)])

In [ ]:
pd.DataFrame(history.history).plot()
plt.title("Loss")
plt.show()

In [ ]:
def word_for_id(integer, tokenizer):
    # map an integer to a word
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None
 
def predict_seq(model, tokenizer, source):
    # generate target from a source sequence
    prediction = model.predict(source, verbose=0)[0]
    integers = [np.argmax(vector) for vector in prediction]
    target = list()
    for i in integers:
        word = word_for_id(i, tokenizer)
        if word is None:
            break
        target.append(word)
    return ' '.join(target)

def compare_prediction(model, tokenizer, sources, raw_dataset, limit=20):
    # evaluate a model
    actual, predicted = [], []

    for i, source in enumerate(sources): # translate encoded source text
        source = source.reshape((1, source.shape[0]))
        translation = predict_seq(model, tar_tokenizer, source)
        raw_target, raw_src = dev_en['translation_output'][i], dev_tcn['text'][i]
        print(f'{raw_src:30} {raw_target:25} {translation}')
        if i >= limit: # Display some of the result
            break

            
print('### Result on the Training Set ###')
compare_prediction(model, tar_tokenizer, trainX, train)

print('\n\n### Result on the Test Set ###')
compare_prediction(model, tar_tokenizer, testX, test)

In [ ]:
import sacrebleu

# It takes long to compute the BLEU Score
def bleu_score(model, tokenizer, sources, raw_dataset):
    # Get the bleu score of a model
    actual, predicted = [], []
    for i, source in enumerate(sources):
        # translate encoded source text
        source = source.reshape((1, source.shape[0]))
        translation = predict_seq(model, tar_tokenizer, source)
        raw_target, raw_src = raw_dataset.iloc[i, 0], raw_dataset.iloc[i, 1]
        actual.append([raw_target.split()])
        predicted.append(translation.split())
        
    bleu = sacrebleu.corpus_bleu(actual, predicted)
    
    return bleu.precisions

# Compute the BLEU Score
bleu_train = bleu_score(model, tar_tokenizer, trainX, train)
bleu_test = bleu_score(model, tar_tokenizer, testX, test)